In [1]:
import sys 
import os 

sys.path.append('../..')

DATA_DIR = '../../data'
os.environ['DATA_DIR'] = DATA_DIR

from utils.correlation import calc_correlation, calc_correlation2
from utils.data import load_data, load_gpt, load_cslb, load_sorting, load_cslb_count_vec, load_mcrae, generate_concepts_to_keep, match_behv_sim, load_behav
from utils.vectorization import get_all_vectorized, get_all_vectorized2
import matplotlib.pyplot as plt 

# Load data 

In [ ]:
group_to_one_concept = True
duplicates = True 
min_amount_runs_feature_occured = 4
min_amount_runs_feature_occured_within_concept = 1

feature_norms = {
    'McRae': load_mcrae(group_to_one_concept, duplicates),
    'CSLB': load_cslb(group_to_one_concept),
    'GPT': load_gpt(min_amount_runs_feature_occured, group_to_one_concept, min_amount_runs_feature_occured_within_concept, duplicates)
}

behav_sim = load_behav()

In [2]:
feature_norms_vec = get_all_vectorized2(feature_norms, behav_sim, 'count')

../../data


/home/hannes/anaconda3/envs/features/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/hannes/anaconda3/envs/features/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Amount of concepts to keep: 317


# Predicting human similarity judgements

## THINGS

In [3]:
corr = calc_correlation2(feature_norms_vec, behav_sim)
corr.style.background_gradient(cmap='coolwarm')

,THINGS,MCRae,CSLB,GPT
THINGS,1.000000,0.559284,0.630856,0.621724
MCRae,0.559284,1.000000,0.788641,0.789170
CSLB,0.630856,0.788641,1.000000,0.798895
GPT,0.621724,0.789170,0.798895,1.000000


In [ ]:
r_gpt_behav, r_cslb_behav, r_mc_behav, r_gpt_mc, r_cslb_gpt = calc_correlation(gpt_vec, mc_vec, behv_sim, cslb_vec)

## Model performance with more runs

In [ ]:
min_amount_runs_feature_occured = 1
group_to_one_concept = False
min_amount_runs_feature_occured_within_concept = 1
duplicates = True 

gpt_df = load_gpt(min_amount_runs_feature_occured, group_to_one_concept, min_amount_runs_feature_occured_within_concept, duplicates)
mc_df = load_mcrae(True)
clsb_df = load_cslb(True)
        
r = []
for i in range(1, 31):
    runs = list(range(1, i+1))
    print(runs)
    gpt_df_temp = gpt_df[gpt_df['run_nr'].isin(runs)]
    gpt_df_temp = gpt_df_temp.groupby('concept_id', as_index=False).agg({'feature': lambda x: ';'.join(x)})
    
    gpt_vec, cslb_vec, mc_vec, behv_sim_matched = get_all_vectorized(gpt_df_temp, clsb_df, mc_df, behv_sim, 'binary')
    r_gpt_behav, r_cslb_behav, r_mc_behav, r_gpt_mc, r_cslb_gpt = calc_correlation(gpt_vec, mc_vec, behv_sim_matched, cslb_vec)
    r.append(r_gpt_behav)

In [ ]:
plt.plot(range(1, len(r) + 1), r)